# Load libraries and data

In [ ]:
library(ANCOMBC)
library(tidyverse)
library(phyloseq)
library(ggrepel)
library(viridis)
library(microbiome)
library(pheatmap)

options(repr.plot.width=15, repr.plot.height=10)

In [ ]:
nasal_phylo_raw <- readRDS("../../results/Microbiome_analysis/nasal_samples_clean_raw.rds")
nasal_phylo_raw

In [ ]:
gut_phylo_raw <- readRDS("../../results/Microbiome_analysis/gut_samples_clean_raw.rds")
gut_phylo_raw

# Functions

In [ ]:
filter_data <- function(ps, prev_threshold = 0.10, rel_abund_threshold = 0.01) {

	# Calculate prevalence of each ASV
  prevdf <- apply(
		X = otu_table(ps),
		MARGIN = ifelse(taxa_are_rows(ps), 1, 2),
		FUN = function(x) sum(x > 0)
	)

  # Calculate relative abundance of each ASV
  total_counts <- sum(taxa_sums(ps))
  rel_abundance <- taxa_sums(ps) / total_counts * 100

  # Add taxonomy, prevalence, and relative abundance to data.frame
  prevdf <- data.frame(
		Prevalence = prevdf,
		RelativeAbundance = rel_abundance,
		tax_table(ps)
	)

  # Define prevalence threshold as % of total samples
  prevalenceThreshold <- prev_threshold * nsamples(ps)

  # Execute prevalence and relative abundance filter, using `prune_taxa()` function
  keepTaxa <- rownames(prevdf)[(
		prevdf$Prevalence >= prevalenceThreshold &
		prevdf$RelativeAbundance >= rel_abund_threshold
	)]

  ps_filtered <- prune_taxa(keepTaxa, ps)

  # Calculate removed and retained ASVs
  removed_asvs <- ntaxa(ps) - ntaxa(ps_filtered)
  retained_asvs <- ntaxa(ps_filtered)

  # Print results
  cat("Original number of ASVs:", ntaxa(ps), "\n")
  cat("Number of ASVs removed:", removed_asvs, "\n")
  cat("Number of ASVs retained:", retained_asvs, "\n")
  cat("Prevalence threshold:", prev_threshold, "\n")
  cat("Relative abundance threshold:", rel_abund_threshold, "% \n")

  return(ps_filtered)
}


In [ ]:
perform_ancombc2 <- function(data, fixed_formula, rand_formula = NULL, group, em_max_iter = 100, em_tol = 1e-5) {

  result <- ancombc2(
    data = data,                 # Input data
    assay_name = "counts",       # Name of the assay containing count data
    tax_level = "Genus",         # Taxonomic level for analysis
    fix_formula = fixed_formula, # Fixed effects formula
    rand_formula = rand_formula, # Random effects formula
    p_adj_method = "BH",         # Method for p-value adjustment (Benjamini-Hochberg)
    pseudo_sens = TRUE,          # Perform pseudo-count sensitivity analysis
    s0_perc = 0.05,              # Percentile for calculating prior variance
    group = group,               # Grouping variable
    struc_zero = TRUE,           # Account for structural zeros
    neg_lb = TRUE,               # Use negative lower bound for bias correction
    alpha = 0.05,                # Significance level
    n_cl = 12,                   # Use 12 cores 
    verbose = FALSE,              # Print progress messages
    global = FALSE,              # Perform global test
    pairwise = TRUE,             # Perform pairwise comparisons
    dunnet = TRUE,               # Perform Dunnett's test
    trend = FALSE,               # Perform trend test
    mdfdr_control = list(        # Control parameters for MDFDR
      fwer_ctrl_method = "BH",   # FWER control method
      B = 1000                   # Number of permutations
    ),
    em_control = list(           # Control parameters for EM algorithm
      tol = em_tol,              # Convergence tolerance
      max_iter = em_max_iter     # Maximum number of iterations
    )
  )
  return(result)
}

In [ ]:
subset_treatment_timepoints <- function(phylo_obj, treatments, genera_of_interest, timepoints) {
  # Get sample data
  sample_data <- data.frame(sample_data(phylo_obj))
  
  # Filter samples for the specified treatments and timepoints
  samples_to_keep <- rownames(sample_data)[
		sample_data$treatment %in% treatments & 
		sample_data$timepoint %in% timepoints
	]
  
  # Prune samples
  phylo_filtered <- prune_samples(samples_to_keep, phylo_obj)
  
  # Get patients present in all specified timepoints
  samples_df <- data.frame(sample_data(phylo_filtered))
  
  samples_at_all_timepoints <- samples_df %>%
    group_by(patient) %>%
    summarize(timepoint_count = n_distinct(timepoint)) %>%
    filter(timepoint_count == length(timepoints)) %>%
    pull(patient)
  
  if(length(samples_at_all_timepoints) == 0) {
    stop("No patients found with data at all specified timepoints")
  }
  
  # Further filter for patients with all timepoints
  samples_to_keep_final <- rownames(samples_df)[samples_df$patient %in% samples_at_all_timepoints]
  phylo_filtered <- prune_samples(samples_to_keep_final, phylo_filtered)
    
  return(phylo_filtered)
}

In [ ]:
plot_ancomb_results_timepoint_healthy <- function(ancomb_results, title) {

	process_ancombc_results_timepoint <- function(ancomb_results) {
		processed_df <- ancomb_results$res  %>%
			# First filter out lines ending with NA
			filter(!endsWith(taxon, "NA"))  %>%
			# Keep everything after the fifth underscore
			mutate(taxon = sub("^([^_]*_){5}", "", taxon)) %>%
			filter(
				diff_timepoint0 == TRUE | 
				diff_timepoint28 == TRUE | 
				diff_timepoint90 == TRUE |
				diff_timepoint180 == TRUE
				) %>%
			mutate(
				lfc_timepoint0 = ifelse(diff_timepoint0 == FALSE, 0, lfc_timepoint0),
				lfc_timepoint28 = ifelse(diff_timepoint28 == FALSE, 0, lfc_timepoint28),
				lfc_timepoint90 = ifelse(diff_timepoint90 == FALSE, 0, lfc_timepoint90),
				lfc_timepoint180 = ifelse(diff_timepoint180 == FALSE, 0, lfc_timepoint180),
				) %>%
			mutate(across(where(is.numeric), ~round(., 2))) 

		return(processed_df)
	}

	create_heatmap_timepoints <- function(df, title) {
		# Define new column names
		new_column_names <- c(
			"Dupilumab day 0 vs healthy day 0 " = "lfc_timepoint0",
			"Dupilumab day 28 vs healthy day 0 " = "lfc_timepoint28",
			"Dupilumab day 90 vs healthy day 0 " = "lfc_timepoint90", 
			"Dupilumab day 180 vs healthy day 0 " = "lfc_timepoint180"
		)

		# Prepare the data for heatmap
		heatmap_data <- df %>%
			select(taxon, lfc_timepoint0, lfc_timepoint28, lfc_timepoint90, lfc_timepoint180) %>%
			column_to_rownames("taxon") %>%
			as.matrix()

		# Rename columns
		colnames(heatmap_data) <- names(new_column_names)[match(colnames(heatmap_data), new_column_names)]

		# Calculate the maximum absolute value for symmetrical color scaling
		max_abs_value <- max(abs(heatmap_data), na.rm = TRUE)

		# Create a matrix of cell labels (the LFC values)
		cell_labels <- matrix(sprintf("%.2f", heatmap_data), ncol = ncol(heatmap_data))

		# Create a color matrix for the numbers
		number_colors <- df %>%
			select(taxon, passed_ss_timepoint0, passed_ss_timepoint28, passed_ss_timepoint90, passed_ss_timepoint180) %>%
			column_to_rownames("taxon") %>%
			mutate(across(everything(), ~ifelse(., "white", "black"))) %>%
			as.matrix()

		# Rename columns of number_colors
		colnames(number_colors) <- names(new_column_names)[match(colnames(number_colors), new_column_names)]

		# Perform clustering to get the correct order
		if (nrow(heatmap_data) > 1) {
			clustering <- hclust(dist(heatmap_data))
			ordered_taxa <- rownames(heatmap_data)[clustering$order]
			number_colors <- number_colors[ordered_taxa, ]
		}

		# Create the heatmap
		pheatmap(
			heatmap_data,
			color = viridis(100, option = "H", direction = 1),
			breaks = seq(-max_abs_value, max_abs_value, length.out = 101),
			cluster_rows = nrow(heatmap_data) > 1,
			cluster_cols = FALSE,
			main = title,
			fontsize = 20, 
			angle_col = 0,
			fontsize_row = 25,
			fontsize_col = 25,
			display_numbers = cell_labels,
			number_color = number_colors,
			fontsize_number = 30
		)
	}
    
	df <- process_ancombc_results_timepoint(ancomb_results)


	if (nrow(df) == 0) {
		return("No siginifcant changes detected")
	} else {
		plot <- create_heatmap_timepoints(df, title)
		return(plot)
	} 
}

# Prevalence and abundance filtering

In [ ]:
nasal_phylo_filtered <- filter_data(nasal_phylo_raw, prev_threshold = 0.1, rel_abund_threshold  = 0.01)

In [ ]:
gut_phylo_filtered <- filter_data(gut_phylo_raw, prev_threshold = 0.1, rel_abund_threshold  = 0.01)

# Healthy day 0 compared to various dipilumab timepoints

## Nasal samples

In [ ]:
nasal_phylo_dupilumab <- subset_samples(nasal_phylo_filtered, treatment == "Dupilumab_treatment")

nasal_phylo_healthy_dupi <- subset_treatment_timepoints(
	phylo_obj = nasal_phylo_raw,
	treatment = c("healthy_control"),
	genera_of_interest = genera_of_interest,
	timepoints = c("0")
)

# Update sample data for healthy controls
sample_data(nasal_phylo_healthy_dupi)$timepoint <- "healthy day 0"


# Merge healthy controls with the existing dupilumab filtered data
nasal_phylo_comp_modified <- merge_phyloseq(nasal_phylo_dupilumab, nasal_phylo_healthy_dupi)

# Update sample data for the merged phyloseq object
sample_data(nasal_phylo_comp_modified)$timepoint <- factor(
  sample_data(nasal_phylo_comp_modified)$timepoint,
  levels = c("healthy day 0", "0", "28", "90", "180")
)

nasal_phylo_comp_modified

In [ ]:
nasal_phylo_comp_modified_ancombc2 <- perform_ancombc2(
	nasal_phylo_comp_modified,
	fixed_formula = "timepoint",
	rand_formula = "(1 | patient)",
	group = "timepoint"
)

saveRDS(nasal_phylo_comp_modified_ancombc2,"../../results/Microbiome_analysis/ANCOM-BC2/nasal_phylo_comp_modified_ancombc2.rds")

In [ ]:
nasal_phylo_comp_modified_ancombc2 <- readRDS("../../results/Microbiome_analysis/ANCOM-BC2/nasal_phylo_comp_modified_ancombc2.rds")

In [ ]:
options(repr.plot.width=30, repr.plot.height=25)

nasal_phylo_comp_modified_ancombc2_heatmap <- plot_ancomb_results_timepoint_healthy(nasal_phylo_comp_modified_ancombc2, "Dupilumab treatment accross different timepoints comapred to healthy day 0 (nasal passage samples)")
nasal_phylo_comp_modified_ancombc2_heatmap

## Gut samples

In [ ]:
gut_phylo_dupilumab <- subset_samples(gut_phylo_filtered, treatment == "Dupilumab_treatment")

gut_phylo_healthy_dupi <- subset_treatment_timepoints(
	phylo_obj = gut_phylo_raw,
	treatment = c("healthy_control"),
	genera_of_interest = genera_of_interest,
	timepoints = c("0")
)

# Update sample data for healthy controls
sample_data(gut_phylo_healthy_dupi)$timepoint <- "healthy day 0"


# Merge healthy controls with the existing dupilumab filtered data
gut_phylo_comp_modified <- merge_phyloseq(gut_phylo_dupilumab, gut_phylo_healthy_dupi)

# Update sample data for the merged phyloseq object
sample_data(gut_phylo_comp_modified)$timepoint <- factor(
  sample_data(gut_phylo_comp_modified)$timepoint,
  levels = c("healthy day 0", "0", "28", "90", "180")
)

gut_phylo_comp_modified

In [ ]:
gut_phylo_comp_modified_ancombc2 <- perform_ancombc2(
	gut_phylo_comp_modified,
	fixed_formula = "timepoint",
	rand_formula = "(1 | patient)",
	group = "timepoint"
)

saveRDS(gut_phylo_comp_modified_ancombc2,"../../results/Microbiome_analysis/ANCOM-BC2/gut_phylo_comp_modified_ancombc2.rds")

In [ ]:
gut_phylo_comp_modified_ancombc2 <- readRDS("../../results/Microbiome_analysis/ANCOM-BC2/gut_phylo_comp_modified_ancombc2.rds")

In [ ]:
options(repr.plot.width=30, repr.plot.height=5)

gut_phylo_comp_modified_ancombc2_heatmap <- plot_ancomb_results_timepoint_healthy(gut_phylo_comp_modified_ancombc2, "Dupilumab treatment accross different timepoints comapred to healthy day 0 (gut samples)")
gut_phylo_comp_modified_ancombc2_heatmap

# Saving plots

In [ ]:
# Specify the path where you want to save the plots
save_path <- "../../results/Microbiome_analysis/plots/ANCOM-BC2"

# Create the directory if it doesn't exist
if (!dir.exists(save_path)) {
  dir.create(save_path, recursive = TRUE)
}

# Set plot dimensions and DPI
plot_width <- 30
plot_height <- 25
plot_dpi <- 300

# Get all variables ending with "_plot"
plot_vars <- ls(pattern = "_heatmap$")

# Function to save a plot as both PNG and PDF
save_plots <- function(plot_name) {
  plot_obj <- get(plot_name)
  
  # Save as PNG
  png_filename <- file.path(save_path, paste0(plot_name, ".png"))
  pdf_filename <- file.path(save_path, paste0(plot_name, ".pdf"))
  
  # Check if it's a ggplot object
  if (inherits(plot_obj, "ggplot")) {
    ggsave(png_filename, plot = plot_obj, width = plot_width, height = plot_height, dpi = plot_dpi, units = "in")
    ggsave(pdf_filename, plot = plot_obj, width = plot_width, height = plot_height, units = "in")
  } else {
    # Assume it's a base R plot
    png(png_filename, width = plot_width, height = plot_height, units = "in", res = plot_dpi)
    print(plot_obj)
    dev.off()
    
    pdf(pdf_filename, width = plot_width, height = plot_height)
    print(plot_obj)
    dev.off()
  }
  
  cat("Saved:", png_filename, "\n")
  cat("Saved:", pdf_filename, "\n")
}

# Save each plot
invisible(sapply(plot_vars, save_plots))

cat("Saved", length(plot_vars), "plots (both PNG and PDF) in", save_path, "\n")
cat("Plot dimensions:", plot_width, "x", plot_height, "inches, DPI:", plot_dpi, "\n")